This notebook performs the searches of PubMed and the CORD-19 Database for text representing trial registrations.

# Setup

In [1]:
import sys
from pathlib import Path
import os
cwd = os.getcwd()
parent = str(Path(cwd).parents[0])
sys.path.append(parent)

%load_ext autoreload
%autoreload 2

In [2]:
import os
from tqdm.auto import tqdm
import re
import json
import pandas as pd
import numpy as np
import xmltodict
import tarfile
import pickle

from bs4 import BeautifulSoup
from xml.etree.ElementTree import tostring

#Our searching function and lists of our regular expressions
from lib.id_searches import search_text, ids_exact, prefixes, registry_names

# PubMed Search

In [3]:
#If the archive exists, load it in.
try:
    from lib.id_searches import zip_load
    archive_df = zip_load(parent + '/data/pubmed/pubmed_archive_1Jan_2021.csv.zip', 
                  'pubmed_archive_1Jan_2021.csv', index_col = 0)

#If it doesn't exist, you can do a new PubMed search
except FileNotFoundError:
    from pymed import PubMed
    from lib.credentials import email
    from lib.id_searches import query, create_pubmed_archive
    print('Archive file not found, conduting new PubMed search.')
    pubmed = PubMed(tool="Pymed", email=email)
    results = pubmed.query(query, max_results=150000)
    results_length = pubmed.getTotalResultsCount(query)
    print(f'There are {results_length} results')
          
    print('Transforming results. This may take a few minutes.')
    #results_list = list(results) #This can take a while
    #print(f'Transformed {len(results_list)} results')
    
    #archive_df = create_pubmed_archive(results_list)
    archive_df = create_pubmed_archive(results, results_length)
    archive_df.to_csv(parent + '/data/pubmed/pubmed_archive_1Jan_2021.csv')
    print('Archive created')
    

In [5]:
pubmed_data = archive_df.xml_json.tolist()

In [6]:
pubmed_dicts = []
for rec in tqdm(pubmed_data):
    pm_dict = json.loads(rec)['PubmedArticle']
    entry_dict = {}
    art_ids = pm_dict['PubmedData']['ArticleIdList']['ArticleId']
    entry_dict['source'] = 'PubMed'
    entry_dict['pmid'] = pm_dict['MedlineCitation']['PMID']['#text']
    entry_dict['doi'] = None
    
    if isinstance(pm_dict['MedlineCitation']['Article']['ArticleTitle'], str):
        entry_dict['title'] = pm_dict['MedlineCitation']['Article']['ArticleTitle']
    elif isinstance(pm_dict['MedlineCitation']['Article']['ArticleTitle'], dict):
        entry_dict['title'] = pm_dict['MedlineCitation']['Article']['ArticleTitle']['#text']
    elif pm_dict['MedlineCitation']['Article']['ArticleTitle'] is None:
        entry_dict['title'] = 'No Title'
    
    if isinstance(art_ids, list):
        for x in art_ids:
            if x['@IdType'] == 'doi':
                entry_dict['doi'] = x['#text']
    elif isinstance(art_ids, dict):
        if art_ids['@IdType'] == 'doi':
            entry_dict['doi'] = art_ids['#text']
    try:
        dbs =  pm_dict['MedlineCitation']['Article']['DataBankList']['DataBank']
        accession_nums = []
        if isinstance(dbs, list):
            for x in dbs:
                ans = x['AccessionNumberList']['AccessionNumber']
                if isinstance(ans, list):
                    accession_nums += ans
                else:
                    accession_nums.append(x)
        elif isinstance(dbs, dict):
            x = dbs['AccessionNumberList']['AccessionNumber']
            if isinstance(x, list):
                accession_nums += x
            else:
                accession_nums.append(x)
                
        if accession_nums:
            entry_dict['accession'] = accession_nums
        else:
            entry_dict['accession'] = None
    except KeyError:
        entry_dict['accession'] = None
        
    
    try:
        pub_type_list = []
        pub_types = pm_dict['MedlineCitation']['Article']['PublicationTypeList']['PublicationType']
        if isinstance(pub_types, list):
            for pt in pub_types:
                pub_type_list.append(pt['#text'])
        elif isinstance(pub_types, dict):
            pub_type_list.append(pub_types['#text'])
        entry_dict['pub_types'] = pub_type_list
    except KeyError:
        entry_dict['pub_types'] = None
    
    try:
        entry_dict['pub_types_raw'] = pm_dict['MedlineCitation']['Article']['PublicationTypeList']
    except KeyError:
        entry_dict['pub_types_raw'] = None
    
    
    try:
        entry_dict['abstract'] = str(pm_dict['MedlineCitation']['Article']['Abstract']['AbstractText'])
    except KeyError:
        entry_dict['abstract'] = None

    pubmed_dicts.append(entry_dict)

In [7]:
for d in tqdm(pubmed_dicts):
    d['abst_id_hits'] = search_text(ids_exact, d['abstract'])
    d['reg_prefix_hits'] = search_text(prefixes, d['abstract'])
    d['reg_name_hits'] = search_text(registry_names, d['abstract'])
    if 'review' in d['title'].lower():
        d['review_in_title'] = True
    else:
        d['review_in_title'] = False
    try:
        if 'Review' in d['pub_types']:
            d['review_type'] = True
        else:
            d['review_type'] = False
    except TypeError:
        d['review_type'] = False

In [8]:
pubmed_search_results = pd.DataFrame(pubmed_dicts)
pubmed_search_results.to_csv(parent + '/data/pubmed/pubmed_search_results_jan2021.csv')

In [9]:
del archive_df

# Searching CORD-19 data

In [10]:
metadata = zip_load(parent + '/data/cord_19/metadata.csv.zip', 'metadata.csv', low_memory = False)
metadata['publish_time'] = pd.to_datetime(metadata['publish_time'])

In [11]:
#Getting a list of all the filenames for the papers that were published in 2020
covid_19_arts = metadata[metadata.publish_time >= pd.Timestamp(2020,1,1)].sha.to_list()
covid_19_pmc = metadata[metadata.publish_time >= pd.Timestamp(2020,1,1)].pmcid.to_list()
recent_articles = []
for c in covid_19_arts:
    recent_articles.append(str(c) + '.json')

recent_pmcs = []
for p in covid_19_pmc:
    recent_pmcs.append(str(p) + '.xml.json')

In [12]:
cord_hit_list = []
with tarfile.open(parent+'/data/cord_19/document_parses.tar.gz', 'r:gz') as tar:
    file_names = tar.getnames()
    files = tar.getmembers()
    for fn, f in zip(file_names, tqdm(files)):
        name = fn.split('/')[-1]
        doc_dict = {}
        if name in recent_articles:
            file = tar.extractfile(f)
            content = file.read()
            stringify = content.decode('utf-8')
            j = json.loads(content)
            doc_dict['file_name'] = j['paper_id']
            doc_dict['source'] = 'cord_pdf'
            doc_dict['id_hits'] = search_text(ids_exact, stringify)
            doc_dict['reg_prefix_hits'] = search_text(prefixes, stringify)
            doc_dict['reg_name_hits'] = search_text(registry_names, stringify)
            doc_dict['title'] = j['metadata']['title']
            if 'review' in doc_dict['title'].lower():
                doc_dict['review_in_title'] = True
            else:
                doc_dict['review_in_title'] = False
        elif name in recent_pmcs:
            file = tar.extractfile(f)
            content = file.read()
            stringify = content.decode('utf-8')
            j = json.loads(content)
            doc_dict['file_name'] = j['paper_id']
            doc_dict['source'] = 'cord_pmc'
            doc_dict['id_hits'] = search_text(ids_exact, stringify)
            doc_dict['reg_prefix_hits'] = search_text(prefixes, stringify)
            doc_dict['reg_name_hits'] = search_text(registry_names, stringify)
            doc_dict['title'] = j['metadata']['title']
            if 'review' in doc_dict['title'].lower():
                doc_dict['review_in_title'] = True
            else:
                doc_dict['review_in_title'] = False
        else:
            continue
        if doc_dict:
            cord_hit_list.append(doc_dict)

In [15]:
cord_df = pd.DataFrame(cord_hit_list)

In [16]:
import pickle
pickle.dump(cord_df, open('cord_df.pkl', 'wb'))

# Bringing it all together

In [72]:
from lib.id_searches import make_doi_url, trial_pub_type, dedupe_list

In [73]:
cord_pdf_merge = cord_df.merge(metadata[['sha', 'doi', 'pubmed_id', 'cord_uid', 'url']], how='left', left_on='file_name', right_on='sha').drop(['sha'], axis=1)

cord_pmc_merge = cord_df.merge(metadata[['pmcid', 'doi', 'pubmed_id', 'cord_uid', 'url']], how='left', left_on='file_name', right_on='pmcid').drop(['pmcid'], axis=1)

In [74]:
combined = cord_pdf_merge[cord_pdf_merge.doi.notnull()].append(cord_pmc_merge[cord_pmc_merge.doi.notnull()], ignore_index=True)

final_cord = combined.loc[combined.astype(str).drop_duplicates().index].drop(['file_name'], axis=1).reset_index(drop=True)

merged = final_cord.merge(pubmed_search_results, how='outer', left_on='pubmed_id', right_on='pmid').drop(['abstract', 'pub_types_raw'], axis=1)

In [75]:
source_conds = [(merged.source_x.notnull() & merged.source_y.isna()), 
                (merged.source_x.isna() & merged.source_y.notnull()), 
                merged.source_x.notnull() & merged.source_y.notnull()]
source_output = [merged.source_x, merged.source_y, 'mixed']

merged['source'] = np.select(source_conds, source_output, None)
merged.drop(['source_x', 'source_y'], axis=1, inplace=True)

In [76]:
title_conds = [(merged.title_x.notnull() & merged.title_y.isna()), 
                (merged.title_x.isna() & merged.title_y.notnull()), 
                merged.title_x.notnull() & merged.title_y.notnull()]
title_output = [merged.title_x, merged.title_y, merged.title_y]

merged['title'] = np.select(title_conds, title_output, None)
merged.drop(['title_x', 'title_y'], axis=1, inplace=True)

In [77]:
doi_conds = [(merged.doi_x.notnull() & merged.doi_y.isna()), 
                (merged.doi_x.isna() & merged.doi_y.notnull()), 
                merged.doi_x.notnull() & merged.doi_y.notnull()]
doi_output = [merged.doi_x, merged.doi_y, merged.doi_y]

merged['doi'] = np.select(doi_conds, doi_output, None)
merged.drop(['doi_x', 'doi_y'], axis=1, inplace=True)

In [78]:
pmid_conds = [(merged.pubmed_id.notnull() & merged.pmid.isna()), 
                (merged.pubmed_id.isna() & merged.pmid.notnull()), 
                merged.pubmed_id.notnull() & merged.pmid.notnull()]
pmid_output = [merged.pubmed_id, merged.pmid, merged.pmid]

merged['pm_id'] = np.select(pmid_conds, pmid_output, None)
merged.drop(['pubmed_id', 'pmid'], axis=1, inplace=True)

In [79]:
review_title_conds = [(merged.review_in_title_x.notnull() & merged.review_in_title_y.isna()), 
                      (merged.review_in_title_x.isna() & merged.review_in_title_y.notnull()), 
                      (merged.review_in_title_x == True) | (merged.review_in_title_y == True), 
                      (merged.review_in_title_x == False) & (merged.review_in_title_y == False)]
review_title_output = [merged.review_in_title_x, merged.review_in_title_y, True, False]

merged['review_in_title'] = np.select(review_title_conds, review_title_output)
merged.drop(['review_in_title_x', 'review_in_title_y'], axis=1, inplace=True)

In [80]:
id_hits_cond = [merged.id_hits.notnull() & merged.abst_id_hits.isna(), 
                merged.id_hits.isna() & merged.abst_id_hits.notnull(), 
                merged.id_hits.notnull() & merged.abst_id_hits.notnull()]
id_hits_output= [merged.id_hits, merged.abst_id_hits, merged.id_hits + merged.abst_id_hits]
    
    
merged['trial_id_hits'] = np.select(id_hits_cond, id_hits_output, None)
merged['trial_id_hits'] = merged['trial_id_hits'].apply(dedupe_list)
merged.drop(['id_hits', 'abst_id_hits'], axis=1, inplace=True)

In [81]:
prefix_hits_cond = [merged.reg_prefix_hits_x.notnull() & merged.reg_prefix_hits_y.isna(), 
                merged.reg_prefix_hits_x.isna() & merged.reg_prefix_hits_y.notnull(), 
                merged.reg_prefix_hits_x.notnull() & merged.reg_prefix_hits_y.notnull()]
prefix_hits_output= [merged.reg_prefix_hits_x, merged.reg_prefix_hits_y, 
                     merged.reg_prefix_hits_x + merged.reg_prefix_hits_y]
    
    
merged['prefix_hits'] = np.select(prefix_hits_cond, prefix_hits_output, None)
merged['prefix_hits'] = merged['prefix_hits'].apply(dedupe_list)
merged.drop(['reg_prefix_hits_x', 'reg_prefix_hits_y'], axis=1, inplace=True)

In [82]:
reg_name_hits_cond = [merged.reg_name_hits_x.notnull() & merged.reg_name_hits_y.isna(), 
                      merged.reg_name_hits_x.isna() & merged.reg_name_hits_y.notnull(), 
                      merged.reg_name_hits_x.notnull() & merged.reg_name_hits_y.notnull()]
reg_name_hits_output= [merged.reg_name_hits_x, merged.reg_name_hits_y, 
                       merged.reg_name_hits_x + merged.reg_name_hits_y]
    
    
merged['reg_name_hits'] = np.select(reg_name_hits_cond, reg_name_hits_output, None)
merged['reg_name_hits'] = merged['reg_name_hits'].apply(dedupe_list)
merged.drop(['reg_name_hits_x', 'reg_name_hits_y'], axis=1, inplace=True)

In [83]:
merged['id'] = np.where(merged.pm_id.isna(), merged.cord_uid, merged.pm_id)

In [84]:
final_df = merged.fillna(np.nan).groupby('id', as_index=False).first()

In [85]:
final_df = final_df[['id', 'trial_id_hits', 'prefix_hits', 'reg_name_hits', 'pub_types', 'doi', 'pm_id', 
                     'cord_uid', 'url', 'title', 'review_in_title', 'review_type']]

final_df.columns = ['id', 'id_hits', 'prefix_hits', 'reg_name_hits', 'pub_types', 'doi', 'pm_id', 'cord_id', 'url', 
                    'title', 'review_in_title', 'review_type']

In [86]:
final_df['pub_types'] = final_df.pub_types.astype(str).apply(trial_pub_type)
final_df['doi_link'] = final_df.doi.apply(make_doi_url)

In [89]:
final_df[((final_df.id_hits.notnull()) | (final_df.pub_types.notnull())) & 
         ((final_df.review_in_title != True) & (final_df.review_in_title != True))]

,id,id_hits,prefix_hits,reg_name_hits,pub_types,doi,pm_id,cord_id,url,title,review_in_title,review_type,doi_link
12,00qqpkfd,[NCT04412304],[NCT],[Clinicaltrials.gov],NaN,10.1101/2020.09.17.20195867,NaN,00qqpkfd,http://medrxiv.org/cgi/content/short/2020.09.1...,DOSING OF THROMBOPROPHYLAXIS AND MORTALITY IN ...,False,NaN,http://doi.org/10.1101/2020.09.17.20195867
24,01hxgcds,"[NCT04328441, NCT04327206]",[NCT],NaN,NaN,10.1101/2020.06.09.20126631,NaN,01hxgcds,https://doi.org/10.1101/2020.06.09.20126631; h...,Rapid characterization of the propagation of C...,False,NaN,http://doi.org/10.1101/2020.06.09.20126631
68,0441ezu8,[NCT04368728],[NCT],NaN,NaN,10.1101/2020.09.08.280818,NaN,0441ezu8,https://doi.org/10.1101/2020.09.08.280818,A prefusion SARS-CoV-2 spike RNA vaccine is hi...,False,NaN,http://doi.org/10.1101/2020.09.08.280818
75,04dq9b4r,[NCT04394416],[NCT],[clinicaltrials.gov],NaN,10.1101/2020.07.07.20148395,NaN,04dq9b4r,https://doi.org/10.1101/2020.07.07.20148395; h...,Disease severity-specific neutrophil signature...,False,NaN,http://doi.org/10.1101/2020.07.07.20148395
207,0cmsi1qh,[ISRCTN16912075],[ISRCTN],NaN,NaN,10.1177/1751143720980280,NaN,0cmsi1qh,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,The variation of FiO 2 with circuit type and p...,False,NaN,http://doi.org/10.1177/1751143720980280
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123085,zsoyupfm,[NCT01363856],[NCT],[ClinicalTrials.gov],NaN,10.1101/2020.04.15.20066324,NaN,zsoyupfm,http://medrxiv.org/cgi/content/short/2020.04.1...,Serum anti-NMDA-receptor antibodies and cognit...,False,NaN,http://doi.org/10.1101/2020.04.15.20066324
123107,zu6xmuql,[NCT04438629],[NCT],[ClinicalTrials.gov],NaN,10.1101/2020.06.26.20135319,NaN,zu6xmuql,http://medrxiv.org/cgi/content/short/2020.06.2...,Baricitinib restrains the immune dysregulation...,False,NaN,http://doi.org/10.1101/2020.06.26.20135319
123123,zvabpty9,[NCT04325646],[NCT],[ClinicalTrials.gov],NaN,10.1101/2020.06.25.20140178,NaN,zvabpty9,https://doi.org/10.1101/2020.06.25.20140178; h...,SARS-CoV-2 infection in primary schools in nor...,False,NaN,http://doi.org/10.1101/2020.06.25.20140178
123141,zwftmuj3,[ChiCTR2000029765],[ChiCTR],NaN,NaN,10.1093/nsr/nwaa041,NaN,zwftmuj3,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PERSPECTIVE IMMUNOLOGY Pathogenic T cells and ...,False,NaN,http://doi.org/10.1093/nsr/nwaa041


In [87]:
final_df[((final_df.id_hits.notnull()) | (final_df.prefix_hits.notnull()) | (final_df.reg_name_hits.notnull()) | (final_df.pub_types.notnull())) & 
         ((final_df.review_in_title != True) & (final_df.review_in_title != True))]

,id,id_hits,prefix_hits,reg_name_hits,pub_types,doi,pm_id,cord_id,url,title,review_in_title,review_type,doi_link
12,00qqpkfd,[NCT04412304],[NCT],[Clinicaltrials.gov],NaN,10.1101/2020.09.17.20195867,NaN,00qqpkfd,http://medrxiv.org/cgi/content/short/2020.09.1...,DOSING OF THROMBOPROPHYLAXIS AND MORTALITY IN ...,False,NaN,http://doi.org/10.1101/2020.09.17.20195867
24,01hxgcds,"[NCT04328441, NCT04327206]",[NCT],NaN,NaN,10.1101/2020.06.09.20126631,NaN,01hxgcds,https://doi.org/10.1101/2020.06.09.20126631; h...,Rapid characterization of the propagation of C...,False,NaN,http://doi.org/10.1101/2020.06.09.20126631
68,0441ezu8,[NCT04368728],[NCT],NaN,NaN,10.1101/2020.09.08.280818,NaN,0441ezu8,https://doi.org/10.1101/2020.09.08.280818,A prefusion SARS-CoV-2 spike RNA vaccine is hi...,False,NaN,http://doi.org/10.1101/2020.09.08.280818
75,04dq9b4r,[NCT04394416],[NCT],[clinicaltrials.gov],NaN,10.1101/2020.07.07.20148395,NaN,04dq9b4r,https://doi.org/10.1101/2020.07.07.20148395; h...,Disease severity-specific neutrophil signature...,False,NaN,http://doi.org/10.1101/2020.07.07.20148395
78,04occmgy,NaN,[Ctri],NaN,NaN,10.1186/s12889-020-09467-x,NaN,04occmgy,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,Influenza A (H1N1)pdm09 outbreak of unknown so...,False,NaN,http://doi.org/10.1186/s12889-020-09467-x
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123107,zu6xmuql,[NCT04438629],[NCT],[ClinicalTrials.gov],NaN,10.1101/2020.06.26.20135319,NaN,zu6xmuql,http://medrxiv.org/cgi/content/short/2020.06.2...,Baricitinib restrains the immune dysregulation...,False,NaN,http://doi.org/10.1101/2020.06.26.20135319
123123,zvabpty9,[NCT04325646],[NCT],[ClinicalTrials.gov],NaN,10.1101/2020.06.25.20140178,NaN,zvabpty9,https://doi.org/10.1101/2020.06.25.20140178; h...,SARS-CoV-2 infection in primary schools in nor...,False,NaN,http://doi.org/10.1101/2020.06.25.20140178
123141,zwftmuj3,[ChiCTR2000029765],[ChiCTR],NaN,NaN,10.1093/nsr/nwaa041,NaN,zwftmuj3,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PERSPECTIVE IMMUNOLOGY Pathogenic T cells and ...,False,NaN,http://doi.org/10.1093/nsr/nwaa041
123166,zxlhxmsq,NaN,[Ctri],NaN,NaN,10.1101/2020.04.28.20075481,NaN,zxlhxmsq,https://doi.org/10.1101/2020.04.28.20075481; h...,Title: Preparedness and response to Pediatric ...,False,NaN,http://doi.org/10.1101/2020.04.28.20075481


In [88]:
final_df.to_csv(parent + '/data/final_auto_24Feb2021.csv')

In [ ]:
#The CORD-19 document parses are too big to fit in the GitHub repo so you need to download and add locally
path_pre = parent + '/data/cord_19/document_parses/'

pdfs = os.listdir(path_pre + 'pdf_json')
pmc = os.listdir(path_pre + 'pmc_json')

In [ ]:
#Now we can only pull out the recent articles from the pdfs
overlap_pdf = list(set(recent_articles).intersection(set(pdfs)))

overlap_pmc = list(set(recent_pmcs).intersection(set(pmc)))

In [ ]:
#These are searches in the CORD-19 database and take roughly an hour to run both.
from lib.id_searches import search_text, ids_exact, prefixes, registry_names

cord_pdf_list = []

for o in tqdm(overlap_pdf):
    with open(path_pre + 'pdf_json' + '/' + o, 'r') as x:
        doc_dict = {}
        c_text = x.read()
        a = json.loads(c_text)
        doc_dict['file_name'] = a['paper_id']
        doc_dict['source'] = 'cord_pdf'
        doc_dict['id_hits'] = search_text(ids_exact, c_text)
        doc_dict['reg_prefix_hits'] = search_text(prefixes, c_text)
        doc_dict['reg_name_hits'] = search_text(registry_names, c_text)
    cord_pdf_list.append(doc_dict)

In [ ]:
cord_pmc_list = []

for o in tqdm(overlap_pmc):
    with open(path_pre + 'pmc_json' + '/' + o, 'r') as x:
        doc_dict = {}
        c_text = x.read()
        a = json.loads(c_text)
        doc_dict['file_name'] = a['paper_id']
        doc_dict['source'] = 'cord_pmc'
        doc_dict['id_hits'] = search_text(ids_exact, c_text)
        doc_dict['reg_prefix_hits'] = search_text(prefixes, c_text)
        doc_dict['reg_name_hits'] = search_text(registry_names, c_text)
    cord_pmc_list.append(doc_dict)

In [ ]:
cord_pmc_df = pd.DataFrame(cord_pmc_list)

final_pmc = cord_pmc_df.merge(metadata, left_on='file_name', right_on='pmcid', how='left')

In [ ]:
cord_pdf_df = pd.DataFrame(cord_pdf_list)

final_pdf = cord_pdf_df.merge(metadata, left_on='file_name', right_on='sha', how='left')

In [ ]:
col_order = ['id', 'source', 'id_hits', 'reg_prefix_hits', 'reg_name_hits', 'accession', 'pub_types', 'doi', 
             'pubmed_id', 'cord_uid']
col_names = ['id', 'source', 'id_hits', 'prefix_hits', 'reg_name_hits', 'accession', 'pub_types', 'doi', 'pm_id', 'cord_id']

interim_cord = final_pmc.append(final_pdf, ignore_index=True).reset_index(drop=True)

interim_cord['id'] = np.where(interim_cord.pubmed_id.isna(), interim_cord.cord_uid, interim_cord.pubmed_id)
interim_cord['accession'] = None
interim_cord['pub_types'] = None

final_cord = interim_cord[col_order].reset_index(drop=True)
final_cord.columns = col_names

In [ ]:
final_cord.to_csv(parent + '/data/cord_19/cord_19_search_results_jan2021.csv')

# Final Data Management

In [ ]:
combined_dataset = final_cord.append(final_pubmed, ignore_index=True)
combined_dataset.head()

In [ ]:
#need to turn the pub_types column into strings:
from lib.id_searches import stringify

combined_dataset['pub_types'] = combined_dataset.pub_types.apply(stringify)

In [ ]:
filter_1 = combined_dataset.id_hits.notnull()

filter_2 = combined_dataset.prefix_hits.notnull()

filter_3 = combined_dataset.reg_name_hits.notnull()

filter_4 = combined_dataset.accession.notnull() & (combined_dataset.accession != '[]')

filter_5 = combined_dataset.pub_types.str.contains(re.compile('(?i)Trial'))

In [ ]:
filtered = combined_dataset[filter_1 | filter_2 | filter_3 | filter_4 | filter_5].sort_values('id').reset_index(drop=True).fillna(np.nan)

In [ ]:
from lib.id_searches import add_lists, make_doi_url, trial_pub_type

combined = filtered[['id', 'id_hits', 'prefix_hits', 'reg_name_hits', 'accession']].groupby('id').agg(add_lists).merge(
    filtered[['id', 'pub_types']][filtered.pub_types.notnull()], how='left', left_on='id', right_on='id').merge(
    filtered[['id', 'doi', 'pm_id']].drop_duplicates(), how='left', left_on='id', right_on='id').merge(
    filtered[['id','cord_id']][filtered.cord_id.notnull()].drop_duplicates(), how='left', left_on='id', right_on='id')

combined['doi'] = combined.doi.apply(make_doi_url)
combined['pub_types'] = combined.pub_types.apply(trial_pub_type)

In [ ]:
final = combined.merge(metadata[['cord_uid', 'url', 'title']], how='left', left_on='cord_id', right_on='cord_uid').drop('cord_uid', axis=1)

In [ ]:
#Do a final dedupe
final_deduped = final.drop_duplicates('id').reset_index(drop=True)